# Importing Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Flatten, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.applications import MobileNetV2, DenseNet169
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load Pre- Trained Models

In [4]:
input_shape = (224, 224, 3)

mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
densenet_base = DenseNet169(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze Base Models

In [6]:
for layer in mobilenet_base.layers:
    layer.trainable = False

for layer in densenet_base.layers:
    layer.trainable = False

# Building Stacked Model

In [8]:
input_layer = Input(shape=input_shape)

In [9]:
# Process MobileNetV2 output
mobilenet_output = GlobalAveragePooling2D()(mobilenet_base(input_layer))
mobilenet_output = Flatten()(mobilenet_output)

# Process DenseNet169 output
densenet_output = GlobalAveragePooling2D()(densenet_base(input_layer))
densenet_output = Flatten()(densenet_output)

In [10]:
# Merge outputs
merged = Concatenate()([mobilenet_output, densenet_output])

In [11]:
# Adding Additional Layers
x = BatchNormalization()(merged)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

In [12]:
# Final output layer with 2 classes
output_layer = Dense(2, activation='softmax')(x)

In [13]:
stacked_model = Model(inputs=input_layer, outputs=output_layer)

In [14]:
stacked_model.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

# Image Data Generators

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

In [17]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
train_generator = train_datagen.flow_from_directory(
    'D:\\Anime\DeepLearningComplete Computer Vision With GenAI-12 Projects\\[TutsNode.org] - DeepLearningComplete Computer Vision With GenAI-12 Projects\\8. Datasets Part 1 (Till this Point)\\train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    'D:\\Anime\DeepLearningComplete Computer Vision With GenAI-12 Projects\\[TutsNode.org] - DeepLearningComplete Computer Vision With GenAI-12 Projects\\8. Datasets Part 1 (Till this Point)\\val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:8: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:8: SyntaxWarning: invalid escape sequence '\D'
C:\Users\MANISH JAISWAL\AppData\Local\Temp\ipykernel_10128\1953560405.py:2: SyntaxWarning: invalid escape sequence '\D'
  'D:\\Anime\DeepLearningComplete Computer Vision With GenAI-12 Projects\\[TutsNode.org] - DeepLearningComplete Computer Vision With GenAI-12 Projects\\8. Datasets Part 1 (Till this Point)\\train',
C:\Users\MANISH JAISWAL\AppData\Local\Temp\ipykernel_10128\1953560405.py:8: SyntaxWarning: invalid escape sequence '\D'
  'D:\\Anime\DeepLearningComplete Computer Vision With GenAI-12 Projects\\[TutsNode.org] - DeepLearningComplete Computer Vision With GenAI-12 Projects\\8. Datasets Part 1 (Till this Point)\\val',


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


# Training The Model

In [20]:
history = stacked_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10)


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8462 - loss: 0.3941

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 566s 3s/step - accuracy: 0.8465 - loss: 0.3934 - val_accuracy: 0.8750 - val_loss: 0.2567
Epoch 2/10


C:\ProgramData\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.2567
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 472s 3s/step - accuracy: 0.9330 - loss: 0.1766 - val_accuracy: 0.9375 - val_loss: 0.1331
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9375 - val_loss: 0.1331
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 517s 3s/step - accuracy: 0.9436 - loss: 0.1433 - val_accuracy: 0.7500 - val_loss: 0.5252
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 0.5252
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 581s 4s/step - accuracy: 0.9403 - loss: 0.1440 - val_accuracy: 0.9375 - val_loss: 0.1793
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9375 - val_loss: 0.1793
Epoch 9/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 585s 4s/step - accuracy: 0.9510 - lo

In [21]:
val_loss, val_accuracy = stacked_model.evaluate(val_generator)
print(f'Validation Accuracy: {val_accuracy:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0977
Validation Accuracy: 1.00


In [22]:
stacked_model.save('model.keras')

Predicting an Image

In [39]:
from tensorflow.keras.preprocessing import image

class_names = ['Normal', 'Pneumonia']

img_path = 'D:\\Anime\\DeepLearningComplete Computer Vision With GenAI-12 Projects\\[TutsNode.org] - DeepLearningComplete Computer Vision With GenAI-12 Projects\\8. Datasets Part 1 (Till this Point)\\test\\PNEUMONIA\\person26_virus_60.jpeg'  
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

predictions = stacked_model.predict(img_array)
predicted_class = np.argmax(predictions)

print(f'Predicted class: {class_names[predicted_class]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
Predicted class: Pneumonia


In [ ]:

img_path = '"D:\Anime\DeepLearningComplete Computer Vision With GenAI-12 Projects\\[TutsNode.org] - DeepLearningComplete Computer Vision With GenAI-12 Projects\\8. Datasets Part 1 (Till this Point)\\test\\NORMAL\\IM-0022-0001.jpeg'  
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

predictions = stacked_model.predict(img_array)
predicted_class = np.argmax(predictions)

print(f'Predicted class: {class_names[predicted_class]}')
